In [1]:
import pandas as pd
import requests
import io
import logging
import traceback
from typing import Optional, List, Dict
from openpyxl import load_workbook
from datetime import datetime
from openai import OpenAI
import urllib.parse
from msal import ConfidentialClientApplication
import os
import shutil
import json
import urllib.parse

In [ ]:
import pandas as pd
import requests
import time
from msal import ConfidentialClientApplication
import os
from tqdm import tqdm

# ============================================
#             CONFIGURATION
# ============================================
# Configuration details
CONFIG = {
    'client_secret': os.getenv("CLIENT_SECRET"),
    'client_id': os.getenv("CLIENT_ID"),
    'tenant_id': os.getenv("TENANT_ID"),
    'bcc_email': os.getenv("BCC_EMAIL"),
    'drive_id': os.getenv("DRIVE_ID"),
    'site_id': os.getenv("SITE_ID"),
    'file_id': os.getenv("FILE_ID"),
    'hubspot_api_key': os.getenv("HUBSPOT_API_KEY"),  
}


client_id = CONFIG['client_id']
tenant_id = CONFIG['tenant_id']
client_secret = CONFIG['client_secret']
drive_id = CONFIG['drive_id']
file_id = CONFIG['file_id']
hubspot_api_key = CONFIG['hubspot_api_key']
bcc_email = CONFIG['bcc_email']


DEFAULT_EMAIL = """ 
<html>
<body style="color: black; font-family: Aptos, Arial, sans-serif; font-size: 12pt; line-height: 1.5;">
    <p style="margin-bottom: 16px;">Dear {first_name},</p>

    <p style="margin-bottom: 16px;">FuelEU pooling is now trading <strong>around EUR 200</strong>— a cheaper, more flexible way than blending biofuels yourself. With FuelEU penalties priced at 3x that level, the choice is clear. </p>

    <p style="margin-bottom: 16px;">Over the past two quarters, we’ve coordinated some of the first FuelEU pooling deals, offering flexible volumes to accommodate <u>uncertainty around year-end emissions.</u> </p>

    <p style="margin-bottom: 16px;">Here’s how we've supported our clients: </p>
    <ul style="margin-bottom: 16px;">
        <li><strong>Cut compliance costs </strong>via pooling</li>
        <li>Access<strong> flexible volumes</strong> and a <strong>fixed-price </strong></li>
        <li>Connect with <strong>trusted counterparties </strong></li>
    </ul>

    <p style="margin-bottom: 16px;">With 50+ years of combined experience in fuel and environmental commodity trading, we work <i>independently </i>to secure the best deal for our clients.  </p>

    <p style="margin-bottom: 16px;">Would you be open to a quick 15-minute call next week to explore options? If timing’s not right, I can send over a short overview instead. </p>

    <p style="margin-bottom: 16px;">Kind regards,</p>

<!-- Signature starts here -->
    <table cellpadding="0" cellspacing="0" border="0" style="font-family: Arial, sans-serif; font-size: 12px; color: #000; line-height:16px;">
    <tr>
        <td colspan="2" style="padding-bottom: 8px;">
        <strong style="font-size:14px; color:#000;">Teresa Gupta</strong>
        </td>
    </tr>
    <tr>
        <td colspan="2" style="height:5px;"></td> <!-- Spacer -->
    </tr>
    <tr>
        <td style="border-top:1px solid #cccccc; padding-top:10px; padding-right:25px; vertical-align:top;">
        <div style="margin-bottom:6px;">
            Open for a call: 
            <a href="tel:+34660645338" style="color:#0078d4; text-decoration:none;">+34 660645338</a>
        </div>
        <div style="margin-bottom:6px;">
            Send an email: 
            <a href="mailto:teresaindira@carbonleap.nl" style="color:#0078d4; text-decoration:none;">teresaindira@carbonleap.nl</a>
        </div>
        <div>
            Visit our website: 
            <a href="https://www.carbonleap.nl" target="_blank" style="color:#0078d4; text-decoration:none;">www.carbonleap.nl</a>
        </div>
        </td>
        <td style="border-top:1px solid #cccccc; padding-top:10px; vertical-align:top;">
        <img src="https://yourdomain.com/carbonleap-logo.png" alt="CarbonLeap Logo" style="width:100px; display:block; margin-bottom:10px;">
        <div style="color:#000;">Laan van Kronenburg 14</div>
        <div style="color:#000;">1183 AS Amstelveen</div>
        <div style="color:#000;">The Netherlands</div>
        </td>
    </tr>
    </table>


<p style="font-size: 10pt; color: #A0A0A0; margin-top: 20px; font-family: Aptos, Arial, sans-serif;">
    <i>At Carbonleap, we’ve facilitated many of the first FuelEU pooling agreements — connecting ship operators with compliance deficits to vetted surplus holders from our trusted network of suppliers. With a dedicated team on FuelEU and combined 50+ years of experience in fuel and environmental commodity trading, we manage the full process: from sourcing from reliable partners and negotiating fair terms, to administrative and reporting obligations.</i>
</p>

</body>
</html>

"""



def authenticate_client(client_id: str, tenant_id: str, client_secret: str) -> str:
    """
    Authenticate with Microsoft Graph API using Client Credentials Flow.
    Returns access token.
    """
    try:
        authority = f'https://login.microsoftonline.com/{tenant_id}'
        app = ConfidentialClientApplication(
            client_id=client_id,
            authority=authority,
            client_credential=client_secret
        )
        token_response = app.acquire_token_for_client(scopes=['https://graph.microsoft.com/.default'])

        if 'access_token' in token_response:
            print("Successfully acquired access token.")
            return token_response['access_token']
        else:
            raise Exception("Failed to acquire access token.")
    except Exception as e:
        print(f"Authentication failed: {str(e)}")
        raise

def create_draft_email(access_token: str, sender_email: str, recipient_email: str, subject: str, body: str) -> bool:
    """
    Create an email draft in the sender's Microsoft Outlook inbox.
    """
    endpoint = f"https://graph.microsoft.com/v1.0/users/{sender_email}/messages"
    email_msg = {
        "subject": subject,
        "body": {"contentType": "HTML", "content": body},
        "toRecipients": [{"emailAddress": {"address": recipient_email}}],
        "bccRecipients": [{"emailAddress": {"address": bcc_email}}],  # Add BCC here
        "ccRecipients": [{"emailAddress": {"address": cc_email}}],
        "isDraft": True
    }

    headers = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}

    print(f"Sending email draft to Outlook API: {recipient_email} - {subject}")

    response = requests.post(endpoint, headers=headers, json=email_msg)

    print(f"Response Code: {response.status_code}")
    print(f"Response Body: {response.text}")  # Debugging API response

    return response.status_code == 201

def read_excel_data(file_path: str):
    """
    Read contact data from Excel file
    """
    try:
        df = pd.read_excel(file_path)
        required_columns = ["Company Name", "First Name", "Last Name", "Job Title", "Email"]
        
        # Check if all required columns exist
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Error: Missing required columns in Excel file: {missing_columns}")
            return None
            
        # Clean up data
        df = df.dropna(subset=["Email", "First Name"])  # Remove rows with missing emails or first names
        
        print(f"Successfully loaded {len(df)} contacts from Excel file")
        return df
    except Exception as e:
        print(f"Error reading Excel file: {str(e)}")
        return None

def create_personalized_email(row, template=DEFAULT_EMAIL):
    """
    Create personalized email content based on contact info
    """
    try:
        first_name = row["First Name"]
        personalized_email = template.format(first_name=first_name)
        
        
        return personalized_email
    except KeyError as e:
        print(f"Error personalizing email: Missing field {e}")
        return None

def main():
    # Use the sender email you want
    sender_email = "guidolevie@carbonleap.nl"
    cc_email = "meeszniehe@carbonleap.nl"
    
    # Get authentication token
    access_token = authenticate_client(client_id, tenant_id, client_secret)
    
    # Get path to Excel file
    file_path = "Batch1_Guido.xlsx"
    contacts_df = read_excel_data(file_path)
    
    if contacts_df is None or len(contacts_df) == 0:
        print("No valid contacts found. Exiting.")
        return
    
    # Confirm before proceeding
    print(f"Found {len(contacts_df)} contacts. Preview of first 5:")
    print(contacts_df.head())
    
    proceed = input(f"Do you want to create {len(contacts_df)} draft emails? (y/n): ").lower()
    if proceed != 'y':
        print("Operation canceled by user.")
        return
    
    # Process each contact
    success_count = 0
    error_count = 0
    
    for index, row in tqdm(contacts_df.iterrows(), total=len(contacts_df), desc="Creating drafts"):
        try:
            recipient_email = row["Email"]
            company_name = row.get("Company Name", "")
            first_name = row["First Name"]
            last_name = row["Last Name"]
            
            # Create subject line
            subject = f"FuelEU Maritime Compliance - pooling cheaper than blending"
            
            # Create personalized email body
            email_body = create_personalized_email(row)
            
            if email_body:
                # Create draft email
                success = create_draft_email(
                    access_token=access_token,
                    sender_email=sender_email,
                    recipient_email=recipient_email,
                    subject=subject,
                    body=email_body,
                )
                
                if success:
                    success_count += 1
                else:
                    error_count += 1
                    print(f"Failed to create draft for {recipient_email}")
                
                # Add a small delay to avoid rate limiting
                time.sleep(1)
            else:
                error_count += 1
                print(f"Failed to generate email content for {recipient_email}")
                
        except Exception as e:
            error_count += 1
            print(f"Error processing contact {index}: {str(e)}")
    
    print(f"\nCompleted email draft creation:")
    print(f"- Successfully created: {success_count}")
    print(f"- Failed: {error_count}")
    print(f"- Total processed: {success_count + error_count}")

if __name__ == "__main__":
    main()

Successfully acquired access token.
Successfully loaded 25 contacts from Excel file
Found 25 contacts. Preview of first 5:
                                   Company Name First Name  Last Name  \
0                                   Chemship BV     Jeroen  Leenderts   
1                       Candler Schiffahrt GmbH     Martin    Humpert   
2  Atlantska plovidba d.d. - Dubrovnik, Croatia       Miro      Matić   
3                     Astra Shipmanagement Inc.     George    Tolakis   
4                                      XT Group   Rajendra      Pawar   

                            Job Title                    Email  
0  Manager Marine Services Department  j.leenderts@chemship.nl  
1                          Operations     m.humpert@candler.eu  
2                     Quality Manager     miro.matic@atlant.hr  
3       Head Of Operations (Dry Bulk)   gtolakis@astraship.com  
4                    Crew Coordinator  rajendra@xtholdings.com  


Creating drafts:   0%|          | 0/25 [00:00<?, ?it/s]

Sending email draft to Outlook API: j.leenderts@chemship.nl - FuelEU Maritime Compliance - pooling cheaper than blending
Response Code: 201
Response Body: {"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users('guidolevie%40carbonleap.nl')/messages/$entity","@odata.etag":"W/\"CQAAABYAAAB3wSKPr6byRoIIi0HIOCNHAAMSSdiX\"","id":"AAMkADczYjI1MDFiLTU1YzgtNDNlNi04ODUyLWVhMDE0ZjcyMTlmZABGAAAAAACxd-WkVHRFQqRI_f9zej_xBwB3wSKPr6byRoIIi0HIOCNHAAAAAAEPAAB3wSKPr6byRoIIi0HIOCNHAAMTxBRYAAA=","createdDateTime":"2025-04-24T09:28:57Z","lastModifiedDateTime":"2025-04-24T09:28:57Z","changeKey":"CQAAABYAAAB3wSKPr6byRoIIi0HIOCNHAAMSSdiX","categories":[],"receivedDateTime":"2025-04-24T09:28:57Z","sentDateTime":"2025-04-24T09:28:57Z","hasAttachments":false,"internetMessageId":"<AM6PR07MB46140AB97E890FEAA4399B25DE852@AM6PR07MB4614.eurprd07.prod.outlook.com>","subject":"FuelEU Maritime Compliance - pooling cheaper than blending","bodyPreview":"Dear Jeroen,\r\n\r\nFuelEU pooling is now trading around

Creating drafts:   4%|▍         | 1/25 [00:01<00:33,  1.39s/it]

Sending email draft to Outlook API: m.humpert@candler.eu - FuelEU Maritime Compliance - pooling cheaper than blending
Response Code: 201
Response Body: {"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users('guidolevie%40carbonleap.nl')/messages/$entity","@odata.etag":"W/\"CQAAABYAAAB3wSKPr6byRoIIi0HIOCNHAAMSSdiZ\"","id":"AAMkADczYjI1MDFiLTU1YzgtNDNlNi04ODUyLWVhMDE0ZjcyMTlmZABGAAAAAACxd-WkVHRFQqRI_f9zej_xBwB3wSKPr6byRoIIi0HIOCNHAAAAAAEPAAB3wSKPr6byRoIIi0HIOCNHAAMTxBRZAAA=","createdDateTime":"2025-04-24T09:28:59Z","lastModifiedDateTime":"2025-04-24T09:28:59Z","changeKey":"CQAAABYAAAB3wSKPr6byRoIIi0HIOCNHAAMSSdiZ","categories":[],"receivedDateTime":"2025-04-24T09:28:59Z","sentDateTime":"2025-04-24T09:28:59Z","hasAttachments":false,"internetMessageId":"<AM6PR07MB461453E31FB86F6DB8A26515DE852@AM6PR07MB4614.eurprd07.prod.outlook.com>","subject":"FuelEU Maritime Compliance - pooling cheaper than blending","bodyPreview":"Dear Martin,\r\n\r\nFuelEU pooling is now trading around EU

...

Sending email draft to Outlook API: stefanos.xronis@anek.gr - FuelEU Maritime Compliance - pooling cheaper than blending
Response Code: 201
Response Body: {"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users('guidolevie%40carbonleap.nl')/messages/$entity","@odata.etag":"W/\"CQAAABYAAAB3wSKPr6byRoIIi0HIOCNHAAMSSdjF\"","id":"AAMkADczYjI1MDFiLTU1YzgtNDNlNi04ODUyLWVhMDE0ZjcyMTlmZABGAAAAAACxd-WkVHRFQqRI_f9zej_xBwB3wSKPr6byRoIIi0HIOCNHAAAAAAEPAAB3wSKPr6byRoIIi0HIOCNHAAMTxBRvAAA=","createdDateTime":"2025-04-24T09:29:25Z","lastModifiedDateTime":"2025-04-24T09:29:25Z","changeKey":"CQAAABYAAAB3wSKPr6byRoIIi0HIOCNHAAMSSdjF","categories":[],"receivedDateTime":"2025-04-24T09:29:25Z","sentDateTime":"2025-04-24T09:29:25Z","hasAttachments":false,"internetMessageId":"<AM6PR07MB46144D18DB0039ADEF118783DE852@AM6PR07MB4614.eurprd07.prod.outlook.com>","subject":"FuelEU Maritime Compliance - pooling cheaper than blending","bodyPreview":"Dear Stefanos,\r\n\r\nFuelEU pooling is now trading arou

Creating drafts:  96%|█████████▌| 24/25 [00:29<00:01,  1.19s/it]

Sending email draft to Outlook API: ioannis.karagiannis@andriaki.gr - FuelEU Maritime Compliance - pooling cheaper than blending
Response Code: 201
Response Body: {"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users('guidolevie%40carbonleap.nl')/messages/$entity","@odata.etag":"W/\"CQAAABYAAAB3wSKPr6byRoIIi0HIOCNHAAMSSdjH\"","id":"AAMkADczYjI1MDFiLTU1YzgtNDNlNi04ODUyLWVhMDE0ZjcyMTlmZABGAAAAAACxd-WkVHRFQqRI_f9zej_xBwB3wSKPr6byRoIIi0HIOCNHAAAAAAEPAAB3wSKPr6byRoIIi0HIOCNHAAMTxBRwAAA=","createdDateTime":"2025-04-24T09:29:26Z","lastModifiedDateTime":"2025-04-24T09:29:26Z","changeKey":"CQAAABYAAAB3wSKPr6byRoIIi0HIOCNHAAMSSdjH","categories":[],"receivedDateTime":"2025-04-24T09:29:26Z","sentDateTime":"2025-04-24T09:29:26Z","hasAttachments":false,"internetMessageId":"<AM6PR07MB46149F3D9E759251C7272A60DE852@AM6PR07MB4614.eurprd07.prod.outlook.com>","subject":"FuelEU Maritime Compliance - pooling cheaper than blending","bodyPreview":"Dear Ioannis,\r\n\r\nFuelEU pooling is now tradi

Creating drafts: 100%|██████████| 25/25 [00:30<00:00,  1.21s/it]


Completed email draft creation:
- Successfully created: 25
- Failed: 0
- Total processed: 25
